In [1]:
import csv
import multiprocessing 
from multiprocessing import Pool

In [2]:
def subdivizija(G):
    # naredi subdivizijo grafa, na vsako povezavo postavi eno vozlišče
    H = G.copy()
    H.subdivide_edges(G.edges(), 1)
    return H


def reverse_subdivizija(sub_G, st_voz):
    # iz subdivizije grafa G vrne originalni graf G
    # sub_G -> subdivizija G, st_voz -> št vozlišč grafa G

    G = sub_G.copy()
    for u, v, _ in G.edges():
        # preverimo, če je u "novo" vozlišče
        if u > st_voz:
            # shranimo sosede u-ja
            neighbor1, neighbor2 = G.neighbors(u)
            # zbrišemo u, kar zbriše tudi vse njegove povezave
            G.delete_vertex(u)
            # dodamo povezavo med sosedi u-ja, ki so "prvotna" vozlišča
            G.add_edge(neighbor1, neighbor2)

        # enako naredimo še za v
        if v > st_voz:
            neighbor1, neighbor2 = G.neighbors(v)
            G.delete_vertex(v)
            G.add_edge(neighbor1, neighbor2)

    return G

# zgornja funkcija deluje, ker so "nova" vozlišča v subdiviziji označena od števila vozlišč prvotnega grafa naprej in je vsako "novo" vozlišče stopnje 2
# torej moreta biti njegova soseda "prvotna" vozlišča


def generiraj_subkubicne_grafe(n):
    # generira vse subdivizije subkubičnih grafov na n vozliščih
    return [subdivizija(G) for G in graphs.nauty_geng(f"{n} -c -D3")] 


def prikazi_graf(G, pakirno_stevilo, barve_vozlisc):
    # izpiše pakirno število, slovar barv vozlišč in nariše graf

    # slovar barv, ki jih bomo uporabili za risanje grafov
    barve = {1 : "blue", 2 : "green", 3 : "red", 4 : "orange", 5 : "purple", 6 : "gold", 7 : "gray"}
    
    # ustvari slovar barv s katerim bomo pobarvali vozlišča
    slovar_barv = {}
    for v, b in barve_vozlisc.items():
        if barve[b] in slovar_barv.keys():
            slovar_barv[barve[b]].append(v)
        else:
            slovar_barv[barve[b]] = [v]

    plt = G.plot(vertex_color=slovar_barv)
    plt.show()
    print(barve_vozlisc, f"Pakirno kromatično število: {pakirno_stevilo}")


def nalozi_seznam(ime_dat):
    # naloži seznam grafov shranjen v datoteki
    # ime_dat je treba podati kot string
    return load(ime_dat)

In [3]:
def poisci_pakirno_stevilo(G, max_colors, min_packing_number):
    # poišče pakirno število grafa G, če je le to >= min_packing_number, št. barv <= max_colors,
    # vrne trojico: graf G, pakirno število grafa, slovar vozlišč s pripadajočimi barvami, sicer vrne None
    
    V = G.vertices()
    D = G.distance_matrix()
    
    # ustvarimo CLP
    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True) # x_v,i -> vozlišče v je pobarvano z barvo i
    z = p.new_variable(nonnegative=True) # šteje koliko barv smo porabili (pakirno število)
    
    # minimiziramo število uporabljenih barv
    p.set_objective(z[0])
    
    # omejitve:
    # 1) vsako vozlišče je pobarvano z natanko eno barvo
    for v in V:
        p.add_constraint(sum(x[v, i] for i in range(1, max_colors + 1)) == 1)
    
    # 2) če sta vozlišči oddaljeni <i+1 sta pobarvani z različno barvo
    for v in V:
        for u in V:
            for i in range(1, max_colors + 1): # za probat range(1, D[v][u] + 1)
                if (D[v][u] <= i) and (v != u):
                    p.add_constraint(x[v, i] + x[u, i] <= 1)
    
    # 3) barva vsakega vozlišča ni večja od pakirnega števila
    for v in V:
        for i in range(1, max_colors + 1):
            p.add_constraint(i * x[v, i] <= z[0])                

    # reši CLP
    p.solve()
    x_sol = p.get_values(x) 
    z_sol = p.get_values(z[0]) 

    # vrnemo le grafe s pakirnim številom >= min_packing_number
    if z_sol >= min_packing_number:
        # ustvari slovar, ključi so vozlišča, vrednosti so barve vozlišč
        barve_vozlisc = {v: next(i for i in range(1, max_colors + 1) if x_sol[v, i] > 0.5) for v in V}
        return (G, z_sol, barve_vozlisc)
    else:
        return

In [4]:
def odpakiraj(arg):
    # pomožna funkcija za paralelizacijo, iz trojice loči argumente, da jih lahko podamo v funkcijo "poisci_pakirno_stevilo"
    G, max_colors, min_packing_number = arg
    return poisci_pakirno_stevilo(G, max_colors, min_packing_number)


def parallel_pakirno_stevilo(grafi, max_colors, min_packing_number, num_processes=None, chunk_size=None):
    # na grafih iz seznama "grafi" požene CLP za iskanje pakirnega števila, vrne seznam trojic (graf, pakirno število, slovar barv vozlišč)
    # omogoča nam, da uporabimo več jeder procesorja in je tako hitreje
    
    # ustvari seznam (elementi so trojice) ki ga podamo kot argument v pool.map
    arg = [(G, max_colors, min_packing_number) for G in grafi]

    # ustvarimo "multiprocessing pool"
    with Pool(processes=num_processes) as pool:
        # pool.map je kot navadna map funkcija le za te multiproc., funkciji "odpakiraj" poda kot argument vsak element iz seznama "arg"
        rezultat = pool.map(odpakiraj, arg, chunksize=chunk_size) 

    # odstranimo "None" rezultate (grafi s premajhnim pakirnim številom)
    # to je seznam trojic, ki jih vrne funkcija "poisci_pakirno_stevilo"
    rezultat = [r for r in rezultat if r is not None]

    return rezultat

In [ ]:
%%time
# brute force iskanje na n vozliščih, shranimo le grafe s pakirnim številom >= 5

n = 13
grafi = generiraj_subkubicne_grafe(n)  
max_colors = 6
min_packing_number = 5
num_cores = multiprocessing.cpu_count() - 1  # uporabimo vsa jedra procesorja razen enega
chunk_size = 200

rezultat = parallel_pakirno_stevilo(grafi, max_colors, min_packing_number, num_processes=num_cores, chunk_size=chunk_size)

save(rezultat, "grafi_na_13_voz")

In [3]:
def lastnosti_grafa(G, pakirno_st, ime_dat):
    # preveri lastnosti grafa G in jih zapiše (skupaj s pakirnim številom) v csv datoteko
    # ime_dat je treba podati kot string

    dvodelnost = G.is_bipartite()
    ravninski = G.is_planar()
    st_povezav = G.size()
    # koliko povezav ima  od vseh možnih
    if G.order() % 2 == 0:
        gostota = round(st_povezav / (3 * G.order() / 2), 3)
    else:
        gostota = round(st_povezav / (3 * (G.order() - 1) / 2), 3)
    povp_stopnja = round(G.average_degree(), 3)
    # meri koliko se vozlišča "gostijo" skupaj
    zgoščenost = round(G.clustering_average(), 3)
    # povprečna razdalja vseh vozlišč do vseh ostalih
    povp_razd = round(G.average_distance(), 3)
    # dolžina najdaljše poti
    najd_pot = len(G.longest_path().edges(sort=True, labels=False))
    premer = G.diameter()

    podatki = [pakirno_st, dvodelnost, ravninski, st_povezav, gostota, povp_stopnja, zgoščenost, povp_razd, najd_pot, premer]
    
    # imena stolpcev CSV datoteke, samo če datoteka še ne obstaja
    imena_stol = ["Pakirno_st", "Dvodelnost", "Ravninski", "St_povezav", "Gostota", "Povp_stopnja", "Zgoscenost", "Povp_razdalja", "Najdaljsa_pot", "Premer"]

    # preverimo ali datoteka obstaja, če ne dodamo glavo
    try:
        with open(ime_dat, "r") as f:
            datoteka_obstaja = True
    except FileNotFoundError:
        datoteka_obstaja = False

    # zapišemo podatke 
    with open(ime_dat, mode="a", newline="") as f:
        writer = csv.writer(f)
        
        # če datoteka še ne obstaja dodamo glavo
        if not datoteka_obstaja:
            writer.writerow(imena_stol)
        
        # zapišemo novo vrstico
        writer.writerow(podatki)

    return 

In [ ]:
# zapišemo podatke za najdene grafe v csv datoteke

for i in range(8, 14):
    # naloži grafe iz datoteke
    grafi = nalozi_seznam(f"grafi_na_{i}_voz.sobj") 
    
    for sub_G, pakirno_stevilo, barve_vozlisc in grafi:
        # dobimo nazaj originalen graf
        G = reverse_subdivizija(sub_G, i) 

        # izračunamo in zapišemo lastnosti grafa
        lastnosti_grafa(G, pakirno_stevilo, f"lastnosti_na_{i}_voz.csv")

    print(f"ZAPISANO ZA {i} VOZ")     